<a href="https://colab.research.google.com/github/Abhi-kumar-21/3D-NUS-NMR-Reconstruction/blob/main/Dense_U_Net_3D_NUS_NMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
from time import time
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import keras
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn import preprocessing
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv3D, MaxPooling3D
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

#Parallelization library
from multiprocessing import Pool, Manager, Process
import threading

from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/My Drive/Scheduler.xlsx')

arr = df.to_numpy().astype('int')

mul_arr = np.zeros((128,32),'float')
for d in arr:
  mul_arr[d[0],d[1]] = 1

Mounted at /content/drive


In [ ]:
mul_arr

In [ ]:
def make_fid(freqs, freqs1, freqs2, r2s, ints, noise=False):
    
  retval=np.zeros( (NP, NP1, NP2) , dtype=np.complex)
  
  # Time samples
  samp_time = np.linspace(0.,(NP-1.)/SW,NP)
  samp_time1 = np.linspace(0.,(NP1-1.)/SW1,NP1)
  samp_time2 = np.linspace(0.,(NP2-1.)/SW2,NP2)
  
  K = np.max([len(freqs), len(freqs2), len(freqs2), len(r2s), len(ints)])
  for n in range(NP):
    for n1 in range(NP1):
      for n2 in range(NP2):
        val = 0.
        for s in range(K):
          val+= (np.exp((1j*2*np.pi*freqs[s] - r2s[s])*samp_time[n])*
                  np.exp((1j*2*np.pi*freqs1[s] - r2s[s])*samp_time1[n1])*
                  np.exp((1j*2*np.pi*freqs2[s] - r2s[s])*samp_time2[n2])*
                  ints[s])
        retval[n,n1,n2]=val

  return retval


def make_sine_combinations():

  # Number of Signals
  nSignals = random.randint(50,NSignals+1)  
  # print(nSignals)

  # Frequencies
  freqs = np.random.random(nSignals)*SW
  freqs1 = np.random.random(nSignals)*SW1
  freqs2 = np.random.random(nSignals)*SW2

  # Transverse relaxation rate
  r2s = np.random.random(nSignals)*(5) + (1/100)

  #Intensity = amplitude
  ints = np.random.random(nSignals)*600000 + 100000

  # FID creation
  ff = make_fid(freqs,freqs1,freqs2,r2s,ints,NoiseLevel)

  temp_realPart = np.real(ff)
  temp = np.abs(temp_realPart[0,0,0])
  ff = ff/temp

  return ff


def make_2d_mat(b_size):
    
  # Training matrix
  target = np.zeros([b_size*NP, NP1*NP2], dtype=np.float)       # For Target
  train = np.zeros([b_size*NP, NP1, NP2], dtype=np.float)      # For CNN
  train1 = np.zeros([b_size*NP, NP1, NP2], dtype=np.float)     # For test_fft
  train_1 = np.zeros([b_size*NP, NP1*NP2], dtype=np.float)      # For DNN

  # Matix concatenation (training)
  for i in range(b_size):
    ff = make_sine_combinations()
    train_ff = ff

    ff = np.fft.fft(ff,axis=0)
    ff = np.fft.fft(ff,axis=1)
    ff = np.fft.fft(ff,axis=2)

    ff = ff/np.max(np.abs(ff))

    train1[i*NP:(i+1)*NP,:,:] = np.real(ff)
    # train1[i*NP:(i+1)*NP,1,:,:] = np.imag(ff)

    for j in range(NP):
      train_ff[j,:,:] = np.multiply(train_ff[j,:,:],mul_arr)
      # index = (i*(j+1))+j
      index = (i*NP)+j
      # till_index = NP1*NP2
      target[index,:] = np.real(ff[j]).reshape(NP1*NP2)
      # target[index,till_index:2*till_index] = np.imag(ff[j]).reshape(NP1*NP2)
    
    train_ff = np.fft.fft(train_ff,axis=0)
    train_ff = np.fft.fft(train_ff,axis=1)
    train_ff = np.fft.fft(train_ff,axis=2)

    train_ff = train_ff/np.max(np.abs(train_ff))

    train[i*NP:(i+1)*NP,:,:] = np.real(train_ff)
    # train[i*NP:(i+1)*NP,1,:,:] = np.imag(train_ff)
    
    for j in range(NP):
      index = (i*NP)+j
      # till_index = NP1*NP2
      train_1[index,:] = np.real(train_ff[j]).reshape(NP1*NP2)
      # train_1[index,till_index:2*till_index] = np.imag(train_ff[j]).reshape(NP1*NP2)

  return train, train1, target

N = 5000
NP = 2        # Direct dimension
NP1 = 128     # Indirect Dimension 1
NP2 = 32      # Indirect Dimension 1
SW = 1000
SW1 = 10000
SW2 = 4000
NoiseLevel = 0
NSignals = 200
dataIn = []
dataOut = []
epochs = 12
batch_size = 32
val_split = 0.05



# train_r, train_i = make_sine_combinations()
# train, train_test, train_1, target = make_2d_mat(1)
# print(train.shape, target.shape, train_1.shape)

In [ ]:

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization, GlobalMaxPool2D
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU 

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model


In [ ]:
#===============================================================================

# Dense U-Net Network Architecture


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = tf.keras.activations.tanh(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = tf.keras.activations.tanh(x)
    x = BatchNormalization()(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    # inputs = Input(input_shape)

    s1, p1 = encoder_block(input_shape, 16)
    s2, p2 = encoder_block(p1, 32)

    b1 = conv_block(p2, 64)

    d3 = decoder_block(b1, s2, 32)
    d4 = decoder_block(d3, s1, 16)

    outputs = Conv2D(1, 1, padding="same", activation="tanh")(d4)

    return outputs


def densenet(img_shape, f=32):
  repetitions = 6, 12, 24,
  
  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    # x = BatchNormalization()(x)
    x = tf.keras.activations.tanh(x)
    # x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    # x = Conv2D(f, k, strides=s, padding=p)(x)
    x = build_unet(x)

    return x
  
  
  def dense_block(tensor, r):
    for _ in range(r):
      x = bn_rl_conv(tensor, 4*f)
      # x = model = build_unet(input_shape)
      x = bn_rl_conv(x, f, 3)
      tensor = Concatenate()([tensor, x])
    return tensor
  
  
  def transition_block(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=1, padding='same')(x)
    return x
  
  
  input = Input(img_shape)
  
  x = Conv2D(64, 3, strides=1, padding='same')(input)
  x = MaxPool2D(3, strides=1, padding='same')(x)
  
  for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)

  
  output = Conv2D(1, 1, activation='tanh')(d)
  
  model = Model(input, output)

  return model


input_shape = (128, 32, 1)
n_classes = 4096

K.clear_session()
model = densenet(input_shape)
model.summary()

#===============================================================================


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
# model = Sequential()
# model.add(Dense(2, input_dim=1, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:

# def densenet(img_shape, n_classes, f=32):
#   repetitions = 6, 12, 24, 16
  
#   def bn_rl_conv(x, f, k=1, s=1, p='same'):
#     x = BatchNormalization()(x)
#     x = tf.keras.activations.tanh(x)
#     # x = ReLU()(x)
#     x = Conv2D(f, k, strides=s, padding=p)(x)
#     return x
  
  
#   def dense_block(tensor, r):
#     for _ in range(r):
#       x = bn_rl_conv(tensor, 4*f)
#       x = bn_rl_conv(x, f, 3)
#       tensor = Concatenate()([tensor, x])
#     return tensor
  
  
#   def transition_block(x):
#     x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
#     x = AvgPool2D(2, strides=1, padding='same')(x)
#     return x
  
  
#   input = Input(img_shape)
  
#   x = Conv2D(64, 3, strides=1, padding='same')(input)
#   x = MaxPool2D(3, strides=1, padding='same')(x)
  
#   for r in repetitions:
#     d = dense_block(x, r)
#     x = transition_block(d)
  
#   # # x = GlobalAvgPool2D()(d)
  
#   # """ Output """
#   # outputs = Conv2D(1, 1, padding="same", activation="tanh")(x)

#   # model = Model(input, outputs, name="Dense-Net")
#   # return model
# # def densenet(img_shape, n_classes, f=32):
# #   repetitions = 6, 12, 24, 16
  
# #   def bn_rl_conv(x, f, k=1, p='same'):
# #     x = BatchNormalization()(x)
# #     x = tf.keras.activations.tanh(x)
# #     # x = ReLU()(x)
# #     x = Conv2D(f, k, padding=p)(x)
# #     return x
  
  
# #   def dense_block(tensor, r):
# #     for _ in range(r):
# #       x = bn_rl_conv(tensor, 4*f)
# #       x = bn_rl_conv(x, f, 3)
# #       tensor = Concatenate()([tensor, x])
# #     return tensor
  
  
# #   def transition_block(x):
# #     x = bn_rl_conv(x, K.int_shape(x)[-1])
# #     x = AvgPool2D(2, strides = 1, padding='same')(x)
# #     return x
  
  
# #   input = Input(img_shape)
  
# #   x = Conv2D(64, 5 , padding='same')(input)
# #   x = MaxPool2D(2, strides = 1, padding='same')(x)
  
# #   for r in repetitions:
# #     d = dense_block(x, r)
# #     x = transition_block(d)
  
#   # x = GlobalAvgPool2D()(d)
#   # x = Flatten()(d)
  
#   output = Conv2D(1, 1, activation='tanh')(d)
  
#   model = Model(input, output)
#   return model
# #   """ Output """
# #   outputs = Conv2D(1, 1, padding="same", activation="tanh")(x)

# #   model = Model(input, outputs, name="Dense-Net")
# #   return model



# input_shape = 128, 64, 1
# n_classes = 4096

# K.clear_session()
# model = densenet(input_shape, n_classes)
# model.summary()

# Compile the model
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
lr_metric = get_lr_metric(optimizer)

reduce_lr = ReduceLROnPlateau(monitor="loss",
                              factor=0.8,
                              patience=9,
                              min_lr=0.0001
)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
# # def densenet(img_shape, n_classes, f=32):
# #   repetitions = 6, 12, 24
  
# #   def bn_rl_conv(x, f, k=1, s=1, p='same'):
# #     x = BatchNormalization()(x)
# #     x = tf.keras.activations.tanh(x)
# #     # x = ReLU()(x)
# #     x = Conv2D(f, k, strides=s, padding=p)(x)
# #     return x
  
  
# #   def dense_block(tensor, r):
# #     for _ in range(r):
# #       x = bn_rl_conv(tensor, 4*f)
# #       x = bn_rl_conv(x, f, 3)
# #       tensor = Concatenate()([tensor, x])
# #     return tensor
  
  
# #   def transition_block(x):
# #     x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
# #     x = AvgPool2D(2, strides=2, padding='same')(x)
# #     return x
  
  
# #   input = Input(img_shape)
  
# #   x = Conv2D(64, 3, strides=1, padding='same')(input)
# #   x = MaxPool2D(3, strides=1, padding='same')(x)
  
# #   for r in repetitions:
# #     d = dense_block(x, r)
# #     x = transition_block(d)
  
# #   x = GlobalMaxPool2D()(d)
  
# #   output = Dense(n_classes, activation='softmax')(x)
  
# #   model = Model(input, output)
# #   return model

# def unet(input_shape=(572, 572, 1), f=64, steps=3, n_classes=2):
  
#   def downstream(x, f):
#     x = Conv2D(f, 3, activation='relu')(x)
#     d = Conv2D(f, 3, activation='relu')(x)
#     x = MaxPool2D(2, strides=1, padding='same')(d)
#     return d, x
  
  
#   def crop_merge(x, d):
#     _, xw, xh, _ = K.int_shape(x)
#     _, dw, dh, _ = K.int_shape(d)
#     mw, mh = (dw-xw)//2, (dh-xh)//2
    
#     d = Lambda(lambda x: x[:, mw: dw-mw, mh: dh-mh, :])(d)
#     x = Concatenate()([d, x])
#     return x
    
  
#   def upstream(x, f, d):
#     x = UpSampling2D()(x)
#     x = Conv2D(f, 2, padding='same')(x)
#     x = crop_merge(x, d)
#     x = Conv2D(f, 3, activation='relu')(x)
#     x = Conv2D(f, 3, activation='relu')(x)
#     return x
    
  
#   input = Input(input_shape)
#   x = input
  
#   downsampled = []
#   for i in range(steps+1):
#     d, x = downstream(x, f*2**i)
#     downsampled.append(d)
#   x = downsampled.pop()
  
#   for i in range(steps-1, -1, -1):
#     x = upstream(x, f*2**i, downsampled[i])
  
#   output = Conv2D(n_classes, 1)(x)
#   model = Model(input, output)
  
#   return model


# input_shape = 572, 572, 1
# n_classes = 4096

# K.clear_session()
# # model = densenet(input_shape, n_classes)
# model = unet(input_shape, f=64, steps=4, n_classes=2)
# model.summary()

# # Compile the model
# def get_lr_metric(optimizer):
#     def lr(y_true, y_pred):
#         return optimizer.lr
#     return lr

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
# lr_metric = get_lr_metric(optimizer)

# reduce_lr = ReduceLROnPlateau(monitor="loss",
#                               factor=0.8,
#                               patience=9,
#                               min_lr=0.0001
# )
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
def dothing(x1_train, fids, fids_flat, i):
    x1, fids_1, fids_2 = make_2d_mat(16)
    print("yolo")
    # print(x1[0,:,:,:], fids_2[0,:])
    # print(x1.shape, fids_1.shape)
    fids.append(fids_1)
    x1_train.append(x1)
    fids_flat.append(fids_2)

In [ ]:
num_threads = 16

for n_datasets in range(100):
    # print(n_datasets)
    with Manager() as manager:
        # L = manager.list()  # <-- can be shared between processes.
        x1_train = manager.list()
        fids = manager.list()
        fids_flat = manager.list()
        processes = []
        for i in range(num_threads):
            p = Process(target=dothing, args=(x1_train, fids, fids_flat, i))  # Passing the list
            # p = threading.Thread(target=dothing, args=(x1_train, fids, fids_flat, i))
            # print(p)
            # processes.append(p)
            p.start()
            processes.append(p)
        for p in processes:
            p.join()
        x1_train = np.vstack(x1_train)
        fids = np.vstack(fids)
        fids_flat = np.vstack(fids_flat)
      
    print(f"This is the iteration number{n_datasets}")
    print(x1_train.shape, fids.shape)
    print(x1_train[0,:,:], fids[0,:,:])


    X_train, X_test, y_train, y_test = train_test_split(x1_train, fids, test_size=0.2)

    history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=10)

    # history = model.fit(train, fid, epochs=200, batch_size=200)

    loss = np.log10(history.history["loss"])
    val_loss = np.log10(history.history['val_loss'])
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    print(loss)
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    plt.plot(epochs, acc, 'y', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    del x1_train, fids, fids_flat

In [ ]:
ls

drive/  sample_data/


In [ ]:
import os

from google.colab import drive

drive.mount('/content/drive')

os.getcwd()

os.chdir('drive')

os.chdir('MyDrive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

In [ ]:
!mkdir -p saved_model_ud_12_ff
model.save('saved_model_ud_12_ff/my_model_ud_12') 

INFO:tensorflow:Assets written to: saved_model_ud_12_ff/my_model_ud_12/assets


In [ ]:
model = tf.keras.models.load_model('saved_model_ud_12/my_model_ud_12')